In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.utils.np_utils import to_categorical
from sklearn.decomposition import PCA
from keras.optimizers import Adam, SGD, Adadelta, RMSprop, Nadam
import keras.callbacks as kcallbacks
from keras.regularizers import l2
import time
from Utils import zeroPadding, normalization, doPCA, modelStatsRecord, averageAccuracy, ssrn_SS_UP

import collections
from sklearn import metrics, preprocessing
print('Finished importing modules in *SSRN UP* at ' + time.ctime(time.time()))

Finished importing modules in *SSRN UP* at Thu May  2 16:04:10 2019


In [3]:
def indexToAssignment(index_, Row, Col, pad_length):
    new_assign = {}
    for counter, value in enumerate(index_):
        assign_0 = value // Col + pad_length
        assign_1 = value % Col + pad_length
        new_assign[counter] = [assign_0, assign_1]
    return new_assign

print('Finished indexToAssignment module in *SSRN UP* at ' + time.ctime(time.time()))

Finished indexToAssignment module in *SSRN UP* at Thu May  2 16:11:20 2019


In [4]:
def assignmentToIndex( assign_0, assign_1, Row, Col):
    new_index = assign_0 * Col + assign_1
    return new_index

print('Finished assignmentToIndex module in *SSRN UP* at ' + time.ctime(time.time()))

Finished assignmentToIndex module in *SSRN UP* at Thu May  2 16:11:21 2019


In [5]:
def selectNeighboringPatch(matrix, pos_row, pos_col, ex_len):
    selected_rows = matrix[range(pos_row-ex_len,pos_row+ex_len+1), :]
    selected_patch = selected_rows[:, range(pos_col-ex_len, pos_col+ex_len+1)]
    return selected_patch

print('Finished selectNeighboringPatch module in *SSRN UP* at ' + time.ctime(time.time()))

Finished selectNeighboringPatch module in *SSRN UP* at Thu May  2 16:11:22 2019


In [6]:
def sampling(proptionVal, groundTruth):              #divide dataset into train and test datasets
    labels_loc = {}
    train = {}
    test = {}
    m = max(groundTruth)
    for i in range(m):
        indices = [j for j, x in enumerate(groundTruth.ravel().tolist()) if x == i + 1]
        np.random.shuffle(indices)
        labels_loc[i] = indices
        nb_val = int(proptionVal * len(indices))
        train[i] = indices[:-nb_val]
        test[i] = indices[-nb_val:]
#    whole_indices = []
    train_indices = []
    test_indices = []
    for i in range(m):
#        whole_indices += labels_loc[i]
        train_indices += train[i]
        test_indices += test[i]
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    return train_indices, test_indices

print('Finished sampling module in *SSRN UP* at ' + time.ctime(time.time()))

Finished sampling module in *SSRN UP* at Thu May  2 16:11:24 2019


In [7]:
def res4_model_ss():
    model_res4 = ssrn_SS_UP.ResnetBuilder.build_resnet_8((1, img_rows, img_cols, img_channels), nb_classes)

    RMS = RMSprop(lr=0.0003)
    # Let's train the model using RMSprop
    model_res4.compile(loss='categorical_crossentropy', optimizer=RMS, metrics=['accuracy'])

    return model_res4

print('Finished res4 model ss in *SSRN UP* at ' + time.ctime(time.time()))

Finished res4 model ss in *SSRN UP* at Thu May  2 16:11:24 2019


In [11]:
uPavia = sio.loadmat(r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/datasets/UP/PaviaU.mat''')
gt_uPavia = sio.loadmat(r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/datasets/UP/PaviaU_gt.mat''')
data_IN = uPavia['paviaU']
gt_IN = gt_uPavia['paviaU_gt']
print (data_IN.shape)

#new_gt_IN = set_zeros(gt_IN, [1,4,7,9,13,15,16])
new_gt_IN = gt_IN

batch_size = 16
nb_classes = 9
nb_epoch = 200                      #400
img_rows, img_cols = 7, 7           #27, 27
patience = 200

INPUT_DIMENSION_CONV = 103
INPUT_DIMENSION = 103

# 10%:10%:80% data for training, validation and testing

TOTAL_SIZE = 42776
VAL_SIZE = 4281
TRAIN_SIZE = 4281
TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE

img_channels = 103
VALIDATION_SPLIT = 0.90                     # 10% for training and %90 for validation and testing

img_channels = 103
PATCH_LENGTH = 3                #Patch_size (13*2+1)*(13*2+1)

data = data_IN.reshape(np.prod(data_IN.shape[:2]),np.prod(data_IN.shape[2:]))
gt = new_gt_IN.reshape(np.prod(new_gt_IN.shape[:2]),)

data = preprocessing.scale(data)

# scaler = preprocessing.MaxAbsScaler()
# data = scaler.fit_transform(data)

data_ = data.reshape(data_IN.shape[0], data_IN.shape[1],data_IN.shape[2])
whole_data = data_
padded_data = zeroPadding.zeroPadding_3D(whole_data, PATCH_LENGTH)

ITER = 1
CATEGORY = 9

train_data = np.zeros((TRAIN_SIZE, 2*PATCH_LENGTH + 1, 2*PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))
test_data = np.zeros((TEST_SIZE, 2*PATCH_LENGTH + 1, 2*PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))

KAPPA_RES_SS4 = []
OA_RES_SS4 = []
AA_RES_SS4 = []
TRAINING_TIME_RES_SS4 = []
TESTING_TIME_RES_SS4 = []
ELEMENT_ACC_RES_SS4 = np.zeros((ITER, CATEGORY))

#seeds = [1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229]

seeds = [1334]

for index_iter in range(ITER):
    print("# %d Iteration" % (index_iter + 1))
    
    best_weights_RES_path_ss4 = r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/UP_best_RES_3D_SS4_5_''' +  str(index_iter+1) +'.hdf5'

    np.random.seed(seeds[index_iter])
#    train_indices, test_indices = sampleFixNum.samplingFixedNum(TRAIN_NUM, gt)
    train_indices, test_indices = sampling(VALIDATION_SPLIT, gt)

    # TRAIN_SIZE = len(train_indices)
    # print (TRAIN_SIZE)
    #
    # TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE - VAL_SIZE
    # print (TEST_SIZE)

    y_train = gt[train_indices] - 1
    y_train = to_categorical(np.asarray(y_train))

    y_test = gt[test_indices] - 1
    y_test = to_categorical(np.asarray(y_test))

    # print ("Validation data:")
    # collections.Counter(y_test_raw[-VAL_SIZE:])
    # print ("Testing data:")
    # collections.Counter(y_test_raw[:-VAL_SIZE])

    train_assign = indexToAssignment(train_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(train_assign)):
        train_data[i] = selectNeighboringPatch(padded_data,train_assign[i][0],train_assign[i][1],PATCH_LENGTH)

    test_assign = indexToAssignment(test_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(test_assign)):
        test_data[i] = selectNeighboringPatch(padded_data,test_assign[i][0],test_assign[i][1],PATCH_LENGTH)

    x_train = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2],INPUT_DIMENSION_CONV)
    x_test_all = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], INPUT_DIMENSION_CONV)

    x_val = x_test_all[-VAL_SIZE:]
    y_val = y_test[-VAL_SIZE:]

    x_test = x_test_all[:-VAL_SIZE]
    y_test = y_test[:-VAL_SIZE]

    # SS Residual Network 4 with BN
    model_res4_SS_BN = res4_model_ss()

    earlyStopping6 = kcallbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto')
    saveBestModel6 = kcallbacks.ModelCheckpoint(best_weights_RES_path_ss4, monitor='val_loss', verbose=1,
                                                save_best_only=True,
                                                mode='auto')

    tic6 = time.clock()
    print(x_train.shape, x_test.shape)
    history_res4_SS = model_res4_SS_BN.fit(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3], 1), y_train,
        validation_data=(x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], x_val.shape[3], 1), y_val),
        batch_size=batch_size,
        nb_epoch=nb_epoch, shuffle=True, callbacks=[earlyStopping6, saveBestModel6])
    toc6 = time.clock()

    tic7 = time.clock()
    loss_and_metrics_res4_SS = model_res4_SS_BN.evaluate(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1), y_test,
        batch_size=batch_size)
    toc7 = time.clock()

    print('3D RES_SS4 without BN Training Time: ', toc6 - tic6)
    print('3D RES_SS4 without BN Test time:', toc7 - tic7)

    print('3D RES_SS4 without BN Test score:', loss_and_metrics_res4_SS[0])
    print('3D RES_SS4 without BN Test accuracy:', loss_and_metrics_res4_SS[1])

    print(history_res4_SS.history.keys())

    pred_test_res4 = model_res4_SS_BN.predict(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1)).argmax(axis=1)
    collections.Counter(pred_test_res4)
    gt_test = gt[test_indices] - 1
    overall_acc_res4 = metrics.accuracy_score(pred_test_res4, gt_test[:-VAL_SIZE])
    confusion_matrix_res4 = metrics.confusion_matrix(pred_test_res4, gt_test[:-VAL_SIZE])
    each_acc_res4, average_acc_res4 = averageAccuracy.AA_andEachClassAccuracy(confusion_matrix_res4)
    kappa = metrics.cohen_kappa_score(pred_test_res4, gt_test[:-VAL_SIZE])
    KAPPA_RES_SS4.append(kappa)
    OA_RES_SS4.append(overall_acc_res4)
    AA_RES_SS4.append(average_acc_res4)
    TRAINING_TIME_RES_SS4.append(toc6 - tic6)
    TESTING_TIME_RES_SS4.append(toc7 - tic7)
    ELEMENT_ACC_RES_SS4[index_iter, :] = each_acc_res4

    print("3D RESNET_SS4 without BN training finished.")
    print("# %d Iteration" % (index_iter + 1))

modelStatsRecord.outputStats(KAPPA_RES_SS4, OA_RES_SS4, AA_RES_SS4, ELEMENT_ACC_RES_SS4,
                             TRAINING_TIME_RES_SS4,TESTING_TIME_RES_SS4,
                             history_res4_SS, loss_and_metrics_res4_SS, CATEGORY,
                             r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/records/UP_train_SS_10.txt''',
                             r'''C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/records/UP_train_SS_element_10.txt''')

print('Finished *SSRN UP* at ' + time.ctime(time.time()))

(610, 340, 103)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# 1 Iteration
input shape: 103
input shape: (None, 7, 7, 49, 24)
conv_spc_result shape: (None, 7, 7, 1, 128)
conv1 shape: (None, 5, 5, 1, 24)
input shape: (None, 5, 5, 1, 24)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


(4281, 7, 7, 103) (34214, 7, 7, 103)
Train on 4281 samples, validate on 4281 samples
Epoch 1/200
4281/4281 [==============================] - 70s 16ms/step - loss: 1.1947 - acc: 0.6473 - val_loss: 0.4239 - val_acc: 0.9311

Epoch 00001: val_loss improved from inf to 0.42390, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/UP_best_RES_3D_SS4_5_1.hdf5
Epoch 2/200
4281/4281 [==============================] - 64s 15ms/step - loss: 0.7591 - acc: 0.7975 - val_loss: 0.3390 - val_acc: 0.91545

Epoch 00002: val_loss improved from 0.42390 to 0.33902, saving model to C:/Users/josep/Desktop/RS/residualnet_tensorflow_keras/residualnet_tensorflow_keras/SSRN-master/models/UP_best_RES_3D_SS4_5_1.hdf5
Epoch 3/200
4281/4281 [==============================] - 63s 15ms/step - loss: 0.5983 - acc: 0.8496 - val_loss: 0.2746 - val_acc: 0.9362ss: 0 - ETA: 5s - loss: 0.6098 - acc: 0.846 - ETA: 

Epoch 00003: val_loss improved from 0.33902 to 

...

4281/4281 [==============================] - 134s 31ms/step - loss: 0.0903 - acc: 0.9792 - val_loss: 0.0558 - val_acc: 0.9960

Epoch 00188: val_loss did not improve from 0.03234
Epoch 189/200
4281/4281 [==============================] - 129s 30ms/step - loss: 0.0922 - acc: 0.9771 - val_loss: 0.0417 - val_acc: 0.9970

Epoch 00189: val_loss did not improve from 0.03234
Epoch 190/200
4281/4281 [==============================] - 134s 31ms/step - loss: 0.0784 - acc: 0.9792 - val_loss: 0.0420 - val_acc: 0.9967

Epoch 00190: val_loss did not improve from 0.03234
Epoch 191/200
4281/4281 [==============================] - 137s 32ms/step - loss: 0.0782 - acc: 0.9813 - val_loss: 0.0423 - val_acc: 0.9972

Epoch 00191: val_loss did not improve from 0.03234
Epoch 192/200
4281/4281 [==============================] - 154s 36ms/step - loss: 0.0860 - acc: 0.9776 - val_loss: 0.0367 - val_acc: 0.9979

Epoch 00192: val_loss did not improve from 0.03234
Epoch 193/200
4281/4281 [=============================

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:119: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:121: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


34214/34214 [==============================] - 370s 11ms/step
3D RES_SS4 without BN Training Time:  17674.142024900004
3D RES_SS4 without BN Test time: 369.8392129000058
3D RES_SS4 without BN Test score: 0.03332226130733127
3D RES_SS4 without BN Test accuracy: 0.9984216987198223
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:125: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


3D RESNET_SS4 without BN training finished.
# 1 Iteration
Test score: 0.03332226130733127
Test accuracy: 0.9984216987198223
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
Finished *SSRN UP* at Fri May  3 01:31:36 2019
